In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

import glob

from locations import *
from parsing import *

In [2]:
# Idea:
# Más adelante, agrupar las columnas desechadas en una sola para crear una sola llamada 'Descripción'
# y hacer un Cluster con NLP con ella

In [3]:
df = pd.read_csv('madrid_parsed.csv')
df

,price,title,location,lat,lng,Comedor,Referencia,Conservación,Garaje,Ascensor,...,Gas,No se aceptan mascotas,Terraza,Sistema de seguridad,Agua,Adaptado a personas con movilidad reducida,Chimenea,Superficie solar,Cocina,Calle alumbrada
0,176000.0,Apartamento en venta en Abantos-Monte Carmelo-...,Abantos-Monte Carmelo-El Rosario (San Lorenzo ...,40.593445,-4.145386,Se encuentra en la zona del salón,4994318-VF-812,A estrenar,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,116500.0,"Apartamento en venta en Calle de Ercilla, 20, ...","Calle de Ercilla, 20, cerca de Calle del Labra...",40.402079,-3.702151,NaN,EP805-1309,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN
2,169000.0,Apartamento en venta en Ajalvir,Ajalvir,40.534457,-3.479415,NaN,IFC76153-1690TN,En buen estado,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,174000.0,Apartamento en venta en Alcorcón,Casco Antiguo (Alcorcón),40.344358,-3.825283,NaN,SA1398-4340/6592,NaN,NaN,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,130000.0,Apartamento en venta en Alcorcón,Casco Antiguo (Alcorcón),40.347096,-3.827826,NaN,SA2135-04930/7619,NaN,NaN,1,...,NaN,NaN,1,NaN,1,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15832,135000.0,Piso en venta en Calle de la Circunvalación,Calle de la Circunvalación. Zona Suroeste (Tor...,40.454606,-3.455234,NaN,SA4463-BANCO-0001,A reformar,NaN,1,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15833,210000.0,Piso en venta en Calle de Urano,Calle de Urano. Zona Suroeste (Torrejón de Ardoz),40.441928,-3.473036,NaN,SA3697-IL-04891,En buen estado,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15834,324600.0,Piso en venta en Fresnos,Zona Suroeste (Torrejón de Ardoz),40.448403,-3.470899,NaN,SA555-39508999/1670,NaN,1,1,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15835,249900.0,Piso en venta en Calle de Cibeles,Calle de Cibeles. Zona Suroeste (Torrejón de A...,40.444707,-3.473520,NaN,4525678-000021,En buen estado,1,NaN,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
fig = px.bar([df[x].dropna().count() for x in df.columns],
             y = df.columns,
             x = [df[x].dropna().count() for x in df.columns])

fig.update_layout(height = 1000)

In [8]:
len(df['Superficie solar'].unique())

1024

In [18]:
# Lista de columnas a revisar para decidir cómo tratar la información para entrenar el modelo en el futuro
scratched_features = ['Cocina', 'Cocina equipada', 'Superficie solar', 'Sistema de seguridad', 'Portero automático',
                      'Aire acondicionado', 'Puerta blindada', 'Amueblado', 'Trastero', 'Calefacción', 'Gas',
                      'Tipo de casa', 'Piscina', 'Vidrios dobles', 'Planta', 'Tipo suelo', 'Balcón', 'Orientación',
                      'Armarios empotrados', 'Comedor']

# Cocina -> hacerla categórica (americana, independiente, amueblada, sin amueblar)
# Cocina equipada -> revisarla con 'Cocina'
# Superficie solar -> 
# Sistema de seguridad -> hacerla categórica (conserje, alarma, videovigilancia, etc.) mezclándola con 'Portero automático'
# Portero automático -> relacionar con 'Sistema de seguridad'
# Aire acondicionado -> hacerla categórica numérica
# Puerta blindada -> hacerla binaria
# Amueblado -> hacerla categórica (sí, no, parcial)
# Trastero -> hacerla categórica (sí, no, comunitario)
# Calefacción -> hacerla categórica (gas natural, butano, eléctrico, frío-calor, chimenea, no)
# Gas -> hacerla categórica (revisar Calefacción)
# Tipo de casa -> hacerla categórica ('Adosada', 'Pareada', ' Rústica', ' Unifamiliar', 'otro)
# Piscina -> hacerla categórica (privada, comunitaria, opcional-comunitaria, no, etc)
# Vidrios dobles -> hacerla categórica (corredera, oscilobatiente, abrir-cerrar, doble aluminio, no)
# Planta -> hacerla categórica numérica (-2, -1, 0, 1, 2, etc)
# Tipo suelo -> hacerla binaria (frío, caliente)
# Balcón -> hacerla categórica numérica
# Armarios empotrados -> hacerla binaria
# Comedor -> hacerla categórica

In [ ]:
features = ['Habitaciones', 'Superficie construida', 'Jardín', 'Gastos de comunidad', 'Antigüedad', 'Baños',
            'Superficie útil', 'Ascensor', 'Garaje', 'Conservación', 'location', 'lat', 'lng', 'title',
            'agency', 'last updated']
pred = ['price']

# Garaje -> hacer varias columnas columnas (tien o no tiene, cantidad, metros cuadrados)
# location -> sacar columnas (tipo de vía, locación, etc)
# title -> revisar con columna 'Tipo de casa' para obtener una columna con tipo de vivienda